In [1]:
import sys
# sys.path.insert(0, "./")
# sys.path.insert(0, "../../")
import torch
from torch.utils.data import Dataset
from utils import *
import random
import matplotlib.pyplot as plt

from torchvision.models import resnet50, resnet18
import argparse
import traceback
import seaborn as sns
import pandas as pd
import csv
import ast
import pdb
from optmize_wcd_human_exp import minimize_wcd
from utils_human_exp import compute_humal_model_wcd

In [2]:
GRID_SIZE =6
grid_size = GRID_SIZE
time_out = 600
assumed_behavior = "HUMAN"
interval = 335

In [3]:
env_ids = np.array([101, 108,  98,  13, 112,  29,  95,  77, 120,  88,  52, 126,  81,
       137, 100, 115, 136,  72,  14,   6, 130, 104,  76,  78,  83,  62,
        99, 116, 119,  38])
human_model = torch.load(f'models/human_model_grid{grid_size}.pt', map_location=torch.device('cpu'))
def run_optimization(assumed_behavior="OPTIMAL"):
    human_best_lambdas = np.array(pd.read_csv('./data/grid6/HUMAN/HUMAN_BOTH_UNIFORM_test/n_lambdas_17/best_lambdas_6_HUMAN_BOTH_UNIFORM_test.csv',header=None))
    optimal_best_lambdas = np.array(pd.read_csv('./data/grid6/OPTIMAL/OPTIMAL_BOTH_UNIFORM_test/n_lambdas_17/best_lambdas_6_OPTIMAL_BOTH_UNIFORM_test.csv',header = None))
    human_best_lambdas = human_best_lambdas[env_ids]
    optimal_best_lambdas = optimal_best_lambdas[env_ids]
    
    dataset_label = f"data/grid{grid_size}/model_training/dataset_{grid_size}_best.pkl"
    if assumed_behavior=="OPTIMAL":
        model_label = f"models/wcd_nn_model_{grid_size}_best_optimal.pt"
        best_lambdas = optimal_best_lambdas
    else:
        model_label = f"models/wcd_nn_model_{grid_size}_best_human.pt"
        best_lambdas = human_best_lambdas

    with open(dataset_label, "rb") as f:
        loaded_dataset = pickle.load(f)

    
    device ="cuda:0"
    model = torch.load(model_label)
    model = model.to(device).eval()

    for i,k in enumerate(env_ids):
        x, y = loaded_dataset[k*interval]  # Get a specific data sample
        x = x.unsqueeze(0).float().cuda()
        lambdas = eval(best_lambdas[i][-1])
        print(k,k*interval, lambdas)
        # print(human_best_lambdas[i])
        best_x_i, invalid_envs,wcds,true_wcds,x_envs,iters, time_taken, wcd_changes = minimize_wcd(model, 
                                                                                                                       x, 
                                                                                                    lambdas=lambdas,
                                                                max_changes_dist=[1,1] ,
                                                                grid_size =grid_size, max_iter = 25
                                                                                                        )
        print("Final WCD",true_wcds[-1])
        torch.save(x_envs[-1],f"./data/grid6/{assumed_behavior}/final_envs/env_{k*interval}.pt")

In [4]:
run_optimization(assumed_behavior="OPTIMAL")
run_optimization(assumed_behavior="HUMAN")

101 33835 [5, 5]
Pred: 254.5581512451172  True 5
i =  1 Pred: 5.0251665115356445  True 5 time taken 0.32866597175598145
i =  1 Pred: 5.0251665115356445  True 5 time taken 0.32866597175598145
Final WCD 5
108 36180 [5, 1.0]
Pred: 154.72885131835938  True 5
i =  1 Pred: 4.007683277130127  True 4 time taken 0.053327322006225586
i =  3 Pred: 4.008505821228027  True 4 time taken 0.14957046508789062
Final WCD 4
98 32830 [5, 1.0]
Pred: 154.7329559326172  True 5
i =  1 Pred: 5.013175010681152  True 5 time taken 0.08205366134643555
i =  3 Pred: 5.011420249938965  True 4 time taken 0.04386615753173828
Final WCD 4
13 4355 [5, 1.0]
Pred: 155.75013732910156  True 6
i =  1 Pred: 6.031838417053223  True 5 time taken 0.041588783264160156
i =  3 Pred: 6.028599262237549  True 4 time taken 0.03661537170410156
Final WCD 4
112 37520 [5, 5]
Pred: 253.53677368164062  True 4
i =  1 Pred: 4.00378942489624  True 4 time taken 0.17287445068359375
i =  1 Pred: 4.00378942489624  True 4 time taken 0.17287445068359375

In [5]:
48*335

16080

In [6]:
original_envs = []
greedy_true_envs =[]
optimal_model_envs = []
human_model_envs =[]

dataset_label = f"data/grid{grid_size}/model_training/dataset_{grid_size}_best.pkl"
with open(dataset_label, "rb") as f:
    loaded_dataset = pickle.load(f)
        
for env_id in env_ids:
    x,y = loaded_dataset[env_id*interval]
    original_envs.append(x.unsqueeze(0))
    
    x_o=torch.load(f"./data/grid6/OPTIMAL/final_envs/env_{env_id*interval}.pt")
    optimal_model_envs.append(x_o)
    
    x_h=torch.load(f"./data/grid6/HUMAN/final_envs/env_{env_id*interval}.pt")
    human_model_envs.append(x_h)
    
    x_g=torch.load(f"./baselines/data/grid6/timeout_600/HUMAN/BOTH_UNIFORM_GREEDY_TRUE_WCD/final_envs/env_{env_id*interval}_budget19.pt")
    greedy_true_envs.append(x_g)

In [7]:
original_envs = torch.cat(original_envs)
greedy_true_envs = torch.cat(greedy_true_envs)
optimal_model_envs = torch.cat(optimal_model_envs)
human_model_envs = torch.cat(human_model_envs)

In [8]:
# original_envs.shape

In [9]:
original_human_wcds = []
for x_i in original_envs:
    original_human_wcds.append(compute_humal_model_wcd(x_i.cpu(),model =human_model))

In [10]:
optimal_model_human_wcds = []
for x_i in optimal_model_envs:
    optimal_model_human_wcds.append(compute_humal_model_wcd(x_i.cpu(),model =human_model))

In [11]:
data_model_human_wcds = []
for x_i in human_model_envs:
    data_model_human_wcds.append(compute_humal_model_wcd(x_i.cpu(),model =human_model))

In [12]:
greedy_human_wcds = []
for x_i in greedy_true_envs:
    greedy_human_wcds.append(compute_humal_model_wcd(x_i.cpu(),model =human_model))

In [25]:
optimal_wcd_change = np.array(pd.read_csv('./data/grid6/OPTIMAL/OPTIMAL_BOTH_UNIFORM_test/n_lambdas_17/wcd_change_6_OPTIMAL_BOTH_UNIFORM_test.csv',header=None))
human_wcd_change = np.array(pd.read_csv('./data/grid6/HUMAN/HUMAN_BOTH_UNIFORM_test/n_lambdas_17/wcd_change_6_HUMAN_BOTH_UNIFORM_test.csv',header=None))
greedy_wcd_change = np.array(pd.read_csv(f'./baselines/data/grid6/timeout_600/HUMAN/BOTH_UNIFORM_GREEDY_TRUE_WCD/wcd_change_6_BOTH_UNIFORM_GREEDY_TRUE_WCD.csv', header=None))

greedy_wcd_change = greedy_wcd_change[env_ids][:,-1]
optimal_wcd_change = optimal_wcd_change[env_ids][:,-1]
human_wcd_change = human_wcd_change[env_ids][:,-1]

In [27]:
human_wcd_change>=greedy_wcd_change

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True,  True])

In [21]:
human_wcd_change == (np.array(original_human_wcds)-np.array(data_model_human_wcds))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [20]:
optimal_wcd_change == (np.array(original_human_wcds)-np.array(optimal_model_human_wcds))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [28]:
greedy_wcd_change == (np.array(original_human_wcds)-np.array(greedy_human_wcds))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [33]:
torch.save(human_model_envs,"./data/grid6/mturk_validation/our_approach_data_driven.pt")
torch.save(optimal_model_envs,"./data/grid6/mturk_validation/our_approach_optimal_model.pt")
torch.save(greedy_true_envs,"./data/grid6/mturk_validation/greed_true_data_driven.pt")
torch.save(original_envs,"./data/grid6/mturk_validation/original_environments.pt")